In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import keras


In [2]:
def eval_keras(df_name: str, model_name: str, threshold:float) -> list:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # load the saved model
    model = load_model(model_name)

    # read the data for the predictions to be compared to
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the test data and get recommended bets
    winning_bets = []
    losing_bets = []
    for i in range(len(real_X)):
        try:
            prediction = model.predict(scaler.transform(real_X.iloc[i].values.reshape(1, -1)), verbose=0)
            actual = real_y.iloc[i]
            if prediction > threshold and actual == 1:
                winning_bets.append([real_X.iloc[i].loc['Column501'], prediction.item()])
            elif prediction > threshold and actual == 0:
                losing_bets.append([real_X.iloc[i].loc['Column501'], prediction.item()])
        except:
            pass
    return [winning_bets, losing_bets]

In [6]:
import torch
import torch.nn as nn


def eval_pytorch(df_name: str, model_name: str, threshold:float) -> list:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 512)
            self.fc2 = nn.Linear(512, 512)
            self.fc3 = nn.Linear(512, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)

        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.fc3(x)
            return x

    # load the saved model
    model = ANN_Model(input_size=X_train.shape[1])
    model.load_state_dict(torch.load(model_name))
    device = torch.device("cuda")
    model.to(device)
    model.eval()

    # loop through the test data and get recommended bets
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the test data and get recommended bets
    winning_bets = []
    losing_bets = []
    for i in range(len(real_X)):
        X_tensor = torch.tensor(scaler.transform(np.array(real_X.iloc[i].values.reshape(1, -1))), dtype=torch.float32).cuda().to(device)
        y_tensor = torch.tensor([real_y.iloc[i]], dtype=torch.float32).cuda().to(device)
        outputs = model(X_tensor)
        prediction = torch.sigmoid(outputs).item()
        actual = real_y.iloc[i]

        if prediction > threshold and actual == 1:
            winning_bets.append([real_X.iloc[i].loc['Column501'], prediction])
        elif prediction > threshold and actual == 0:
            losing_bets.append([real_X.iloc[i].loc['Column501'], prediction])

    return [winning_bets, losing_bets]


In [4]:
def calculate_winnings(bet_amount, odds):
    if odds > 0:
        return (bet_amount * odds / 100)
    return bet_amount / (odds / -100)


def calculate_profit(bets:list):
    winnings = 0
    losings = 0
    for bet in bets[0]:
        odds = bet[0]
        winnings += calculate_winnings(100, odds)
    for bet in bets[1]:
        losings += calculate_winnings(100, odds)
    return winnings - losings

In [17]:
threshold_list = {}
for i in range(50, 100):
    bets = eval_pytorch("csvs/new_ml_X.csv", "built_models/torch_ml.pth", (i / 100))
    investment = (len(bets[0]) + len(bets[1])) * 100
    profit = calculate_profit(bets) / investment
    threshold_list[i] = profit
    print(i, investment, profit)

50 53900 0.2261244084340613
51 54200 0.2077707562487059
52 53300 0.22337428216817026
53 53100 0.23033234464935415
54 53200 0.21948260278129234
55 51900 0.21085728627054484
56 51300 0.21114953949547186
57 50300 0.2003635661389568
58 50400 0.21137810170533344
59 50000 0.20921169647582988
60 49400 0.19903947947567824
61 48100 0.20924732088362263
62 48300 0.19957206019228915
63 47600 0.11083540232238888
64 47700 0.12415529827204434
65 46900 0.12775262019118713
66 46200 0.11648246274585657
67 46300 0.10977679488195245
68 45500 0.12171920001301967
69 45700 0.11681373146976123
70 44700 0.12252524946664903
71 44100 0.10644364673763244
72 43400 0.10961453644552199
73 42600 0.10840815841320005
74 43200 0.11395110065746115
75 41900 0.10287190391994425
76 42400 0.0967531314680582
77 41500 0.10229806335620527
78 41500 0.10221412965414685
79 41400 0.09884315148279676
80 40300 0.1095016546818213
81 39900 0.09798726621977152
82 39400 0.10055301937031491
83 39600 0.10907062903875617
84 38100 0.09648711

In [16]:
threshold_list = {}
for i in range(50, 100):
    bets = eval_keras("csvs/new_ml_X.csv", "built_models/keras_ml.h5", (i / 100))
    investment = (len(bets[0]) + len(bets[1])) * 100
    profit = calculate_profit(bets) / investment
    threshold_list[i] = profit
    print(i, investment, profit)

50 61100 0.15505435838466708
51 57300 0.15190260311109421
52 52400 0.1718171337510168
53 46400 0.17528736301027947
54 41500 0.1678274904140346
55 36700 0.16813151002353813
56 34000 0.1561394701367648
57 31000 0.14643287857278
58 27700 0.1629642113319983
59 24000 0.16367195521769665
60 21700 0.15664300072771092
61 19200 0.16089115918599683
62 17000 0.1696685892293021
63 15600 0.16700872595500332
64 13800 0.15964852911872607
65 12300 0.129830511602786
66 10200 0.23163307843966982
67 8900 0.20935821186942832
68 7100 0.2086419007191932
69 6500 0.22396100401245853
70 5500 0.29673061106147064
71 4500 0.28990365949159747
72 4300 0.28033050431788126
73 3100 0.14686898326562675
74 3000 0.1287761102940212
75 2500 0.11922237396986325
76 2100 0.05843276704621243
77 2100 0.05843276704621243
78 1700 -0.033847121736395105
79 1600 -0.004712566844919799
80 1400 0.06604278074866309
81 1100 0.15592286501377406
82 800 0.32803030303030295
83 600 0.5383838383838383
84 600 0.5383838383838383
85 600 0.5383838

In [48]:
a = eval_keras("csvs/new_ml_X.csv", "built_models/keras_ml.h5", .9975)
print(len(a[0]), len(a[1]))
print(a)

46 53
[[[165.0, 0.9999998211860657], [155.0, 0.9999998211860657], [-170.0, 0.9999575018882751], [-205.0, 0.9999731779098511], [-185.0, 0.9990999102592468], [-188.0, 0.9991106390953064], [125.0, 0.9998530149459839], [121.0, 0.9998555183410645], [-220.0, 0.9978433847427368], [-240.0, 0.9983752369880676], [-180.0, 0.9999837279319763], [-180.0, 0.9999837279319763], [-170.0, 0.9994417428970337], [-180.0, 0.9994694590568542], [-190.0, 0.9982215166091919], [-190.0, 0.9982249736785889], [-110.0, 0.9998399019241333], [-113.0, 0.9998425841331482], [-225.0, 0.9987900257110596], [-200.0, 0.9985896348953247], [-110.0, 0.9982495903968811], [-106.0, 0.9982390999794006], [-115.0, 0.9999886155128479], [-110.0, 0.9999876022338867], [-150.0, 0.9987389445304871], [-128.0, 0.9984725713729858], [-180.0, 0.9997872710227966], [-162.0, 0.9997240900993347], [-310.0, 0.9998161196708679], [-270.0, 0.999708890914917], [-110.0, 0.9996381998062134], [-111.0, 0.9996432662010193], [-370.0, 0.9999819993972778], [-400.0

In [13]:
b = eval_pytorch("csvs/new_ml_X.csv", "built_models/torch_ml.pth", .9999)
print(len(b[0]), len(b[1]))
print(b)

24 12
[[[165.0, 0.9999984502792358], [155.0, 0.9999985694885254], [-170.0, 0.9999852180480957], [-205.0, 0.9999887943267822], [-190.0, 0.9999169111251831], [-240.0, 0.9999387264251709], [-185.0, 0.9999980926513672], [-164.0, 0.9999977350234985], [-180.0, 0.9999864101409912], [-180.0, 0.9999864101409912], [-160.0, 0.9999995231628418], [-195.0, 0.9999995231628418], [-185.0, 0.9999780654907227], [-156.0, 0.9999693632125854], [-145.0, 0.9999942779541016], [-170.0, 0.9999947547912598], [-315.0, 0.9999990463256836], [-310.0, 0.9999990463256836], [-250.0, 0.9999853372573853], [-240.0, 0.9999845027923584], [-180.0, 0.9999949932098389], [-200.0, 0.9999955892562866], [-370.0, 0.9999616146087646], [-400.0, 0.9999669790267944]], [[-450.0, 0.9999887943267822], [-550.0, 0.9999924898147583], [-280.0, 0.9999752044677734], [-280.0, 0.9999752044677734], [-195.0, 0.9999793767929077], [-185.0, 0.9999784231185913], [-120.0, 0.9999850988388062], [-110.0, 0.9999843835830688], [-300.0, 0.9999704360961914], [-

In [46]:
average_odds = 0
for bet_list in a:
    for bet in bet_list:
        average_odds += bet[0]
average_odds / (len(a[0]) + len(a[1]))

-87.49664429530202

In [14]:
average_odds = 0
for bet_list in b:
    for bet in bet_list:
        average_odds += bet[0]
average_odds / (len(b[0]) + len(b[1]))

-213.47222222222223